# **Waste Material Segregation for Improving Waste Management**

## **Objective**

The objective of this project is to implement an effective waste material segregation system using convolutional neural networks (CNNs) that categorises waste into distinct groups. This process enhances recycling efficiency, minimises environmental pollution, and promotes sustainable waste management practices.

The key goals are:

* Accurately classify waste materials into categories like cardboard, glass, paper, and plastic.
* Improve waste segregation efficiency to support recycling and reduce landfill waste.
* Understand the properties of different waste materials to optimise sorting methods for sustainability.

## **Data Understanding**

The Dataset consists of images of some common waste materials.

1. Food Waste
2. Metal
3. Paper
4. Plastic
5. Other
6. Cardboard
7. Glass


**Data Description**

* The dataset consists of multiple folders, each representing a specific class, such as `Cardboard`, `Food_Waste`, and `Metal`.
* Within each folder, there are images of objects that belong to that category.
* However, these items are not further subcategorised. <br> For instance, the `Food_Waste` folder may contain images of items like coffee grounds, teabags, and fruit peels, without explicitly stating that they are actually coffee grounds or teabags.

## **1. Load the data**

Load and unzip the dataset zip file.

**Import Necessary Libraries**

In [8]:
# Recommended versions:

# numpy version: 1.26.4
# pandas version: 2.2.2
# seaborn version: 0.13.2
# matplotlib version: 3.10.0
# PIL version: 11.1.0
# tensorflow version: 2.18.0
# keras version: 3.8.0
# sklearn version: 1.6.1

In [9]:
# Import essential libraries
!pip install tensorflow
import numpy as mp 
import pandas as pd 
import os
# Deep learning libraries
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
# visualization 
import matplotlib.pyplot as plt
import seaborn as sns

Load the dataset.

In [10]:
# Load and unzip the dataset

import zipfile 
# path to the uploaded zip file
zip_path = '/mnt/data/dataset.zip'
extact_path = '/mnt/data/dataset'

# unzipping the file
with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extact_path)

# verify the directory structure
print("contents:")  
print(os.listdir(extact_path))  

FileNotFoundError: [Errno 2] No such file or directory: '/mnt/data/dataset.zip'

## **2. Data Preparation** <font color=red> [25 marks] </font><br>


### **2.1 Load and Preprocess Images** <font color=red> [8 marks] </font><br>

Let us create a function to load the images first. We can then directly use this function while loading images of the different categories to load and crop them in a single step.

#### **2.1.1** <font color=red> [3 marks] </font><br>
Create a function to load the images.

In [13]:
# Create a function to load the raw images

def load_images_from_directory(base_path, target_size=(128, 128), class_names=None):
    images = []
    labels = []
    
    # If class_names are not specified, infer from directory structure
    if class_names is None:
        class_names = sorted(os.listdir(base_path))
    
    class_to_index = {cls_name: idx for idx, cls_name in enumerate(class_names)}
    
    for cls in class_names:
        class_dir = os.path.join(base_path, cls)
        if not os.path.isdir(class_dir):
            continue
            
        for img_name in os.listdir(class_dir):
            img_path = os.path.join(class_dir, img_name)
            try:
                img = cv2.imread(img_path)
                if img is None:
                    continue
                img = cv2.resize(img, target_size)
                images.append(img)
                labels.append(class_to_index[cls])
            except Exception as e:
                print(f"Error loading image {img_path}: {e}")
    
    images = np.array(images)
    labels = np.array(labels)
    
    return images, labels, class_names



#### **2.1.2** <font color=red> [5 marks] </font><br>
Load images and labels.

Load the images from the dataset directory. Labels of images are present in the subdirectories.

Verify if the images and labels are loaded correctly.

In [ ]:
# Get the images and their labels



Perform any operations, if needed, on the images and labels to get them into the desired format.

### **2.2 Data Visualisation** <font color=red> [9 marks] </font><br>

#### **2.2.1** <font color=red> [3 marks] </font><br>
Create a bar plot to display the class distribution

In [ ]:
# Visualise Data Distribution

# count number of samples per class
label_counts = pd.Series(labels).value_counts().sort_index()
label_names = [class_names[i] for i in label_counts.index]

#plot
plt.figure(figsize=(8,5))
plt.bar(label_names, label_counts, color='skyblue')
plt.xlabel('class')
plt.ylabel('Number of Images ')
plt.title('Image count per class')
plt.xticks(rotation=45)
plt.tight_layout()
plt.show



#### **2.2.2** <font color=red> [3 marks] </font><br>
Visualise some sample images

In [ ]:
# Visualise Sample Images (across different labels)
def show_sample_images(images, labels, class_names, samples_per_class=3):
    plt.figure(figsize=(15, len(class_names) * 2))
    
    for class_index, class_name in enumerate(class_names):
        # Get all indices for this class
        class_indices = [i for i, label in enumerate(labels) if label == class_index]
        selected_indices = random.sample(class_indices, min(samples_per_class, len(class_indices)))
        
        for i, idx in enumerate(selected_indices):
            plt_idx = class_index * samples_per_class + i + 1
            plt.subplot(len(class_names), samples_per_class, plt_idx)
            plt.imshow(images[idx])
            plt.title(class_name)
            plt.axis('off')
    
    plt.tight_layout()
    plt.show()
 show_sample_images(Images, labels, class_names)   


#### **2.2.3** <font color=red> [3 marks] </font><br>
Based on the smallest and largest image dimensions, resize the images.

In [ ]:
# Find the smallest and largest image dimensions from the data set

def find_image_dimensions_extremes(images):
    min_width = float('inf')
    min_height = float('inf')
    min_width = 0
    min_height = 0

    for imag in images:
        heights, width = img.shape[:2]
        if width < min_width:
            min_width = width
        if height < min_height:
            min_height = height
        if width > max_width:
            max_width = width
        if height > max_height:
            max_height = height
print(f"Smallest dimensions: {min_widht}x{min_height}")   
print(f"Largest dimensions: {max_width}x{max_height}")
find_image_dimensions_extremes(images)

In [ ]:
# Resize the image dimensions

def resize_image(image, target_size=(224, 224)):
    resized_images = []
    for img in images:
        resized_image = cv2.resize(img, target_size)
        resized_images.append(resized_image)
    return np.array(resized_images)
resized_images = resize_image(images, target_size=(224, 224))


### **2.3 Encoding the classes** <font color=red> [3 marks] </font><br>

There are seven classes present in the data.

We have extracted the images and their labels, and visualised their distribution. Now, we need to perform encoding on the labels. Encode the labels suitably.

####**2.3.1** <font color=red> [3 marks] </font><br>
Encode the target class labels.

In [ ]:
# Encode the labels suitably

from sklearn.preprocessing import LabelEncoder

label_encoder = LabelEncoder()
integer_encoded_labels = label_encoder.fit_transform(labels)
from tensorflow.keras.utils import to_categorical

one_hot_labels = to_categorical(integer_encoded_labels)


### **2.4 Data Splitting** <font color=red> [5 marks] </font><br>

#### **2.4.1** <font color=red> [5 marks] </font><br>
Split the dataset into training and validation sets

In [ ]:
# Assign specified parts of the dataset to train and validation sets

from sklearn.model_selection import train_test_split

# Assume: `images` is your resized image array, and `one_hot_labels` is the one-hot encoded label array

X_train, X_val, y_train, y_val = train_test_split(
    images, 
    one_hot_labels, 
    test_size=0.2,  # 20% for validation
    random_state=42, 
    stratify=one_hot_labels
)


## **3. Model Building and Evaluation** <font color=red> [20 marks] </font><br>

### **3.1 Model building and training** <font color=red> [15 marks] </font><br>

#### **3.1.1** <font color=red> [10 marks] </font><br>
Build and compile the model. Use 3 convolutional layers. Add suitable normalisation, dropout, and fully connected layers to the model.

Test out different configurations and report the results in conclusions.

In [ ]:
# Build and compile the model

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization

# Define input shape (replace with your resized image dimensions, e.g., 128x128x3)
input_shape = (128, 128, 3)
num_classes = y_train.shape[1]  # e.g., 3 for 3 classes

model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=input_shape),
    BatchNormalization(),
    MaxPooling2D(pool_size=(2, 2)),

    Conv2D(64, (3, 3), activation='relu'),
    BatchNormalization(),
    MaxPooling2D(pool_size=(2, 2)),

    Conv2D(128, (3, 3), activation='relu'),
    BatchNormalization(),
    MaxPooling2D(pool_size=(2, 2)),

    Flatten(),
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(num_classes, activation='softmax')  # Softmax for multi-class classification
])

# Compile the model
model.compile(
    optimizer='adam',
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

model.summary()


#### **3.1.2** <font color=red> [5 marks] </font><br>
Train the model.

Use appropriate metrics and callbacks as needed.

In [ ]:
# Training

# Set training parameters
batch_size = 32
epochs = 15

# Train the model
history = model.fit(
    X_train, y_train,
    batch_size=batch_size,
    epochs=epochs,
    validation_data=(X_val, y_val),
    shuffle=True
)


### **3.2 Model Testing and Evaluation** <font color=red> [5 marks] </font><br>

#### **3.2.1** <font color=red> [5 marks] </font><br>
Evaluate the model on test dataset. Derive appropriate metrics.

In [ ]:
# Evaluate on the test set; display suitable metrics

from sklearn.metrics import classification_report, confusion_matrix
import seaborn as sns
import matplotlib.pyplot as plt

# Predict the labels
y_pred_probs = model.predict(X_test)
y_pred = y_pred_probs.argmax(axis=1)
y_true = y_test.argmax(axis=1)

# Classification report
print("Classification Report:")
print(classification_report(y_true, y_pred, target_names=label_encoder.classes_))

# Confusion matrix
cm = confusion_matrix(y_true, y_pred)

# Plot confusion matrix
plt.figure(figsize=(10, 8))
sns.heatmap(cm, annot=True, fmt="d", cmap="Blues", xticklabels=label_encoder.classes_, yticklabels=label_encoder.classes_)
plt.xlabel("Predicted")
plt.ylabel("True")
plt.title("Confusion Matrix")
plt.show()


## **4. Data Augmentation** <font color=red> [optional] </font><br>

#### **4.1 Create a Data Augmentation Pipeline**

##### **4.1.1**
Define augmentation steps for the datasets.

In [ ]:
# Define augmentation steps to augment images

from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Define augmentation strategy
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.15,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

# For validation/testing, usually only rescaling is applied
test_datagen = ImageDataGenerator(rescale=1./255)


Augment and resample the images.
In case of class imbalance, you can also perform adequate undersampling on the majority class and augment those images to ensure consistency in the input datasets for both classes.

Augment the images.

In [ ]:
# Create a function to augment the images

import tensorflow as tf
from tensorflow.keras import layers

def get_augmentation_model():
    data_augmentation = tf.keras.Sequential([
        layers.RandomFlip("horizontal"),
        layers.RandomRotation(0.1),
        layers.RandomZoom(0.1),
        layers.RandomTranslation(0.1, 0.1),
        layers.RandomContrast(0.1)
    ])
    return data_augmentation



In [ ]:
# Create the augmented training dataset

# First, get the augmentation model
augmenter = get_augmentation_model()

# Define a function to apply augmentation to images and keep labels unchanged
def augment_data(image, label):
    return augmenter(image), label

# Apply the augmentation function to your training dataset
augmented_train_ds = train_ds.map(augment_data, num_parallel_calls=tf.data.AUTOTUNE)

# Optional: Prefetch for performance
augmented_train_ds = augmented_train_ds.prefetch(tf.data.AUTOTUNE)


##### **4.1.2**

Train the model on the new augmented dataset.

In [ ]:
# Train the model using augmented images

# Train the model using the augmented training dataset
history_aug = model.fit(
    augmented_train_ds,
    validation_data=val_ds,
    epochs=EPOCHS
)


## **5. Conclusions** <font color = red> [5 marks]</font>

#### **5.1 Conclude with outcomes and insights gained** <font color =red> [5 marks] </font>

* Report your findings about the data
* Report model training results